In [12]:
import os, subprocess, shutil

# replace this with the nvcc path you posted (we compute CUDA_HOME automatically)
nvcc_path = "/packages/apps/spack/21/opt/spack/linux-rocky8-zen3/gcc-12.1.0/cuda-12.0.1-x3bnvayrybncl3rqu6zk4zzu4oztblqi/bin/nvcc"

if not os.path.exists(nvcc_path):
    raise FileNotFoundError(f"nvcc not found at {nvcc_path} — update nvcc_path if different")

cuda_home = os.path.dirname(os.path.dirname(nvcc_path))
os.environ["CUDA_HOME"] = cuda_home
os.environ["PATH"] = os.path.join(cuda_home, "bin") + os.pathsep + os.environ.get("PATH", "")
# Append existing LD_LIBRARY_PATH if present
os.environ["LD_LIBRARY_PATH"] = os.path.join(cuda_home, "lib64") + os.pathsep + os.environ.get("LD_LIBRARY_PATH", "")

print("CUDA_HOME =", cuda_home)
print("which nvcc ->", shutil.which("nvcc"))

# quick verify (runs in a shell inheriting these env vars)
proc = subprocess.run("/bin/bash -lc 'which nvcc && nvcc --version'", shell=True, capture_output=True, text=True, env=os.environ)
print(proc.stdout)
if proc.returncode != 0:
    print("nvcc failed to run; stderr:\n", proc.stderr)

CUDA_HOME = /packages/apps/spack/21/opt/spack/linux-rocky8-zen3/gcc-12.1.0/cuda-12.0.1-x3bnvayrybncl3rqu6zk4zzu4oztblqi
which nvcc -> /packages/apps/spack/21/opt/spack/linux-rocky8-zen3/gcc-12.1.0/cuda-12.0.1-x3bnvayrybncl3rqu6zk4zzu4oztblqi/bin/nvcc
/packages/apps/spack/21/opt/spack/linux-rocky8-zen3/gcc-12.1.0/cuda-12.0.1-x3bnvayrybncl3rqu6zk4zzu4oztblqi/bin/nvcc
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Fri_Jan__6_16:45:21_PST_2023
Cuda compilation tools, release 12.0, V12.0.140
Build cuda_12.0.r12.0/compiler.32267302_0



In [16]:
%%bash
export CUDA_HOME=/packages/apps/spack/21/opt/spack/linux-rocky8-zen3/gcc-12.1.0/cuda-12.0.1-x3bnvayrybncl3rqu6zk4zzu4oztblqi
export PATH=$CUDA_HOME/bin:$PATH
export LD_LIBRARY_PATH=$CUDA_HOME/lib64:$LD_LIBRARY_PATH
cat > forward_pass.cu <<'EOF'
#include <iostream>
#include <vector>
#include <cmath>
#include <numeric>

// Macro for error checking
#define DIV_UP(a, b) (((a) + (b) - 1) / (b))

#define CUDA_CHECK(call)                                                          \
{                                                                                 \
    cudaError_t err = call;                                                       \
    if (err != cudaSuccess) {                                                     \
        fprintf(stderr, "CUDA error at %s:%d code=%d(%s) \"%s\"\n",               \
                __FILE__, __LINE__, err, cudaGetErrorString(err), #call);         \
        exit(EXIT_FAILURE);                                                       \
    }                                                                             \
}

// Global constants for the network size
const int M = 128;   // Batch Size (Rows)
const int DIN = 64;  // Input Dimension
const int DHID = 32; // Hidden Dimension
const int C = 10;    // Number of Classes (Output Dimension)

const int THREADS_PER_BLOCK = 256;

// --- 1. Initialization and Data Setup Kernels ---

__global__ void initWeightsKernel(float *d_data, int size) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < size) {
        // Use a simple pseudo-random number generation based on thread index
        // In a real application, a more robust RNG like cuRAND would be used
        unsigned int seed = i;
        float rand_val = (float)seed / (float)0xFFFFFFFF; // Scale to 0 to 1
        d_data[i] = (rand_val - 0.5f); // Scale to -0.5 to 0.5
    }
}

// --- 2. Dense Layer (GEMM + Bias Addition) ---

__global__ void simpleGemmBiasKernel(const float *X, const float *W, const float *B, float *Z, 
                                     int M, int K, int N) {
    // Thread index for output matrix Z (row, col)
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < M && col < N) {
        float sum = 0.0f;
        // Perform the dot product of X[row, :] and W[:, col]
        for (int k = 0; k < K; ++k) {
            sum += X[row * K + k] * W[k * N + col];
        }
        
        // Add the bias (B is a 1D vector added to every row)
        sum += B[col];
        
        Z[row * N + col] = sum;
    }
}

// --- 3. Activation Functions ---

__global__ void reluKernel(float *A, int size) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < size) {
        if (A[i] < 0.0f) {
            A[i] = 0.0f;
        }
    }
}

__global__ void softmaxKernel(float *Z, int M, int C) {
    // One thread per row for calculation
    int row = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < M) {
        float max_val = -1e20f;
        float sum_exp = 0.0f;
        int row_start = row * C;
        
        // Find max for numerical stability (simple loop in this simplified kernel)
        for (int j = 0; j < C; ++j) {
            max_val = fmaxf(max_val, Z[row_start + j]);
        }
        
        for (int j = 0; j < C; ++j) {
            Z[row_start + j] = expf(Z[row_start + j] - max_val);
            sum_exp += Z[row_start + j];
        }

        for (int j = 0; j < C; ++j) {
            Z[row_start + j] /= sum_exp;
        }
    }
}

// --- 4. Loss Computation ---

__global__ void crossEntropyKernel(const float *Y_hat, const int *Y_true, float *d_loss_elements, 
                                   int M, int C) {
    // One thread per input sample (row)
    int i = blockIdx.x * blockDim.x + threadIdx.x;

    if (i < M) {
        int true_class = Y_true[i];
        float predicted_prob = Y_hat[i * C + true_class];
        
        float loss = -logf(fmaxf(predicted_prob, 1e-9f)); 
        
        d_loss_elements[i] = loss;
    }
}

/**
 * @brief Performs a generic parallel sum reduction (similar to earlier examples).
 * * @param d_in Input array
 * @param d_out Output array (size 1)
 * @param size Size of the input array
 */
__global__ void reduceSumKernel(float *d_in, float *d_out, int size) {
    // Using a simple block-wise reduction for demonstration
    __shared__ float sdata[THREADS_PER_BLOCK];
    
    int tid = threadIdx.x;
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    
    // Load into shared memory
    sdata[tid] = (i < size) ? d_in[i] : 0.0f;
    __syncthreads();

    // Reduction in shared memory
    for (int s = blockDim.x / 2; s > 0; s >>= 1) {
        if (tid < s) {
            sdata[tid] += sdata[tid + s];
        }
        __syncthreads();
    }
    
    // Write result to global memory (atomic not strictly needed here if grid is small)
    if (tid == 0) {
        // Atomicly add to the final result location (for multi-block summation)
        atomicAdd(d_out, sdata[0]);
    }
}


int main() {
    std::cout << "--- Starting Neural Network Forward Pass on GPU ---" << std::endl;
    std::cout << "M=" << M << ", D_in=" << DIN << ", D_hid=" << DHID << ", C=" << C << std::endl;

    // --- 1. Host Memory Setup ---
    std::vector<int> h_Y_true(M);
    std::vector<float> h_X(M * DIN);
    
    // Initialize Input X (e.g., random data between 0 and 1)
    for (int i = 0; i < M * DIN; ++i) {
        h_X[i] = (float)rand() / RAND_MAX;
    }

    // Initialize True Labels Y_true (e.g., random class indices)
    for (int i = 0; i < M; ++i) {
        h_Y_true[i] = rand() % C;
    }

    // --- 2. Device Memory Allocation ---
    float *d_X, *d_W1, *d_B1, *d_A1; // Layer 1
    float *d_W2, *d_B2, *d_Z2;       // Layer 2 (Output)
    int *d_Y_true;                   // Labels
    float *d_loss_elements, *d_final_loss_sum; // Loss calculation

    // Input and Labels
    CUDA_CHECK(cudaMalloc((void**)&d_X, M * DIN * sizeof(float)));
    CUDA_CHECK(cudaMalloc((void**)&d_Y_true, M * sizeof(int)));

    // Layer 1 Tensors
    CUDA_CHECK(cudaMalloc((void**)&d_W1, DIN * DHID * sizeof(float)));
    CUDA_CHECK(cudaMalloc((void**)&d_B1, DHID * sizeof(float)));
    CUDA_CHECK(cudaMalloc((void**)&d_A1, M * DHID * sizeof(float))); // A1 is Z1 after ReLU

    // Layer 2 Tensors
    CUDA_CHECK(cudaMalloc((void**)&d_W2, DHID * C * sizeof(float)));
    CUDA_CHECK(cudaMalloc((void**)&d_B2, C * sizeof(float)));
    CUDA_CHECK(cudaMalloc((void**)&d_Z2, M * C * sizeof(float))); // Z2 is Softmax input/output

    // Loss Tensors
    CUDA_CHECK(cudaMalloc((void**)&d_loss_elements, M * sizeof(float)));
    CUDA_CHECK(cudaMalloc((void**)&d_final_loss_sum, sizeof(float)));
    
    // --- 3. Initial Transfers (Host -> Device) ---
    CUDA_CHECK(cudaMemcpy(d_X, h_X.data(), M * DIN * sizeof(float), cudaMemcpyHostToDevice));
    CUDA_CHECK(cudaMemcpy(d_Y_true, h_Y_true.data(), M * sizeof(int), cudaMemcpyHostToDevice));
    
    // Initialize final sum to 0
    float zero = 0.0f;
    CUDA_CHECK(cudaMemcpy(d_final_loss_sum, &zero, sizeof(float), cudaMemcpyHostToDevice));

    // --- 4. Launch Initialization Kernels (Weights) ---
    int blocks_W1 = DIV_UP(DIN * DHID, THREADS_PER_BLOCK);
    int blocks_W2 = DIV_UP(DHID * C, THREADS_PER_BLOCK);
    int blocks_B1 = DIV_UP(DHID, THREADS_PER_BLOCK);
    int blocks_B2 = DIV_UP(C, THREADS_PER_BLOCK);

    initWeightsKernel<<<blocks_W1, THREADS_PER_BLOCK>>>(d_W1, DIN * DHID);
    initWeightsKernel<<<blocks_W2, THREADS_PER_BLOCK>>>(d_W2, DHID * C);
    initWeightsKernel<<<blocks_B1, THREADS_PER_BLOCK>>>(d_B1, DHID);
    initWeightsKernel<<<blocks_B2, THREADS_PER_BLOCK>>>(d_B2, C);
        
    // Setup grid for L1 GEMM (M x DHID output)
    dim3 grid1(DIV_UP(DHID, 16), DIV_UP(M, 16)); // Simple 16x16 block decomposition
    dim3 block1(16, 16);

    std::cout << "\n[L1] 1. Executing GEMM + Bias (X * W1 + B1)..." << std::endl;
    // X (M x DIN) * W1 (DIN x DHID) -> A1 (M x DHID)
    simpleGemmBiasKernel<<<grid1, block1>>>(d_X, d_W1, d_B1, d_A1, M, DIN, DHID);

    // Setup grid for L1 ReLU (M * DHID elements)
    int size1 = M * DHID;
    int blocks_relu1 = DIV_UP(size1, THREADS_PER_BLOCK);

    std::cout << "[L1] 2. Executing ReLU Activation..." << std::endl;
    // ReLU(A1) -> A1 (in place)
    reluKernel<<<blocks_relu1, THREADS_PER_BLOCK>>>(d_A1, size1);

    // Setup grid for L2 GEMM (M x C output)
    dim3 grid2(DIV_UP(C, 16), DIV_UP(M, 16));
    dim3 block2(16, 16);

    std::cout << "[L2] 3. Executing GEMM + Bias (A1 * W2 + B2)..." << std::endl;
    simpleGemmBiasKernel<<<grid2, block2>>>(d_A1, d_W2, d_B2, d_Z2, M, DHID, C);

    // Setup grid for Softmax (M rows, 1 block per row)
    int blocks_softmax = DIV_UP(M, THREADS_PER_BLOCK);

    std::cout << "[L2] 4. Executing Softmax Activation..." << std::endl;
    // Softmax(Z2) -> Z2 (in place, now Y_hat)
    softmaxKernel<<<blocks_softmax, THREADS_PER_BLOCK>>>(d_Z2, M, C);

    // --- 7. LOSS CALCULATION ---

    // Setup grid for Cross-Entropy (M samples)
    int blocks_ce = DIV_UP(M, THREADS_PER_BLOCK);

    std::cout << "[Loss] 5. Executing Cross-Entropy (Element-wise Loss)..." << std::endl;
    // CrossEntropy(Y_hat, Y_true) -> d_loss_elements (M x 1)
    crossEntropyKernel<<<blocks_ce, THREADS_PER_BLOCK>>>(d_Z2, d_Y_true, d_loss_elements, M, C);

    // Setup grid for Final Reduction (M elements in d_loss_elements)
    int size_loss = M;
    int blocks_reduce = DIV_UP(size_loss, THREADS_PER_BLOCK);

    std::cout << "[Loss] 6. Executing Reduction (Summing Loss)..." << std::endl;
    // Sum d_loss_elements -> d_final_loss_sum (1 element)
    reduceSumKernel<<<blocks_reduce, THREADS_PER_BLOCK>>>(d_loss_elements, d_final_loss_sum, size_loss);

    // --- 8. Final Transfer and Verification ---
    CUDA_CHECK(cudaDeviceSynchronize()); // Wait for ALL kernels to finish

    float final_loss_sum;
    CUDA_CHECK(cudaMemcpy(&final_loss_sum, d_final_loss_sum, sizeof(float), cudaMemcpyDeviceToHost));
    
    float avg_loss = final_loss_sum / (float)M;

    std::cout << "\n--- Forward Pass Complete ---" << std::endl;
    std::cout << "Total Loss Sum: " << final_loss_sum << std::endl;
    std::cout << "Average Cross-Entropy Loss (Final Output): " << avg_loss << std::endl;
    
    // --- 9. Cleanup ---
    cudaFree(d_X);
    cudaFree(d_Y_true);
    cudaFree(d_W1); cudaFree(d_B1); cudaFree(d_A1);
    cudaFree(d_W2); cudaFree(d_B2); cudaFree(d_Z2);
    cudaFree(d_loss_elements);
    cudaFree(d_final_loss_sum);

    return 0;
}
EOF

nvcc -std=c++17 forward_pass.cu -o forward_pass_output && ./forward_pass_output

--- Starting Neural Network Forward Pass on GPU ---
M=128, D_in=64, D_hid=32, C=10

[L1] 1. Executing GEMM + Bias (X * W1 + B1)...
[L1] 2. Executing ReLU Activation...
[L2] 3. Executing GEMM + Bias (A1 * W2 + B2)...
[L2] 4. Executing Softmax Activation...
[Loss] 5. Executing Cross-Entropy (Element-wise Loss)...
[Loss] 6. Executing Reduction (Summing Loss)...

--- Forward Pass Complete ---
Total Loss Sum: 294.731
Average Cross-Entropy Loss (Final Output): 2.30259


In [17]:
%%bash
export CUDA_HOME=/packages/apps/spack/21/opt/spack/linux-rocky8-zen3/gcc-12.1.0/cuda-12.0.1-x3bnvayrybncl3rqu6zk4zzu4oztblqi
export PATH=$CUDA_HOME/bin:$PATH
export LD_LIBRARY_PATH=$CUDA_HOME/lib64:$LD_LIBRARY_PATH
cat > backward_pass.cu <<'EOF'
#include <iostream>
#include <vector>
#include <cmath>
#include <numeric>

// Global constants for the network size (needed for memory allocation and loop bounds)
const int M = 128;   // Batch Size
const int DIN = 64;  // Input Dimension
const int DHID = 32; // Hidden Dimension
const int C = 10;    // Number of Classes
const float LR = 0.01f; // Learning Rate

const int THREADS_PER_BLOCK = 256;

// Helper macros
#define CUDA_CHECK(call)                                                          \
{                                                                                 \
    cudaError_t err = call;                                                       \
    if (err != cudaSuccess) {                                                     \
        fprintf(stderr, "CUDA error at %s:%d code=%d(%s) \"%s\"\n",               \
                __FILE__, __LINE__, err, cudaGetErrorString(err), #call);         \
        exit(EXIT_FAILURE);                                                       \
    }                                                                             \
}
#define DIV_UP(a, b) ((a + b - 1) / b)

// --- BACKWARD PASS KERNELS ---

/**
 * @brief Calculates the initial gradient dL/dZ2 (pre-softmax/logit).
 * dL/dZ2 = Y_hat - Y_true
 * * @param Y_hat Softmax output (M x C)
 * @param Y_true True labels (M x 1)
 * @param d_dZ2 Output gradient (M x C)
 */
__global__ void lossBackwardKernel(const float *Y_hat, const int *Y_true, float *d_dZ2, 
                                   int M, int C) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < M && col < C) {
        float gradient = Y_hat[row * C + col]; // Start with P_i
        
        // If this column is the true class index, subtract 1
        if (col == Y_true[row]) {
            gradient -= 1.0f;
        }

        d_dZ2[row * C + col] = gradient / (float)M; // Normalize by Batch Size (M)
    }
}

/**
 * @brief Performs the backward pass for Matrix Multiplication: dL/dX = dL/dZ * W^T
 * Calculates the gradient passed back to the previous layer.
 */
__global__ void gemmBackwardInputKernel(const float *d_dZ, const float *W, float *d_dX, 
                                        int M, int K, int N) {
    // dL/dX is (M x K)
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < M && col < K) {
        float sum = 0.0f;
        // Dot product of dZ[row, :] and W^T[:, col]
        for (int n = 0; n < N; ++n) {
            // W is (K x N), W^T access is W[k*N + n] -> W[col*N + n]
            sum += d_dZ[row * N + n] * W[col * N + n];
        }
        d_dX[row * K + col] = sum;
    }
}

/**
 * @brief Performs the backward pass for Matrix Multiplication: dL/dW = X^T * dL/dZ
 * Calculates the gradient of the weights.
 */
__global__ void gemmBackwardWeightKernel(const float *X, const float *d_dZ, float *d_dW, 
                                         int M, int K, int N) {
    // dL/dW is (K x N)
    int row = blockIdx.y * blockDim.y + threadIdx.y; // K rows
    int col = blockIdx.x * blockDim.x + threadIdx.x; // N cols

    if (row < K && col < N) {
        float sum = 0.0f;
        // Dot product of X^T[row, :] and dZ[:, col]
        for (int m = 0; m < M; ++m) {
            // X[m * K + row] * d_dZ[m * N + col]
            sum += X[m * K + row] * d_dZ[m * N + col];
        }
        d_dW[row * N + col] = sum;
    }
}

/**
 * @brief Performs the backward pass for Bias: dL/dB = sum(dL/dZ, axis=0)
 */
__global__ void biasBackwardKernel(const float *d_dZ, float *d_dB, int M, int N) {
    // One thread per column (bias element)
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (col < N) {
        float sum = 0.0f;
        // Sum down the column
        for (int m = 0; m < M; ++m) {
            sum += d_dZ[m * N + col];
        }
        d_dB[col] = sum;
    }
}

/**
 * @brief Performs element-wise ReLU backward: dL/dZ = dL/dA * ReLU'(Z)
 * ReLU'(Z) = 1 if Z > 0, 0 otherwise.
 */
__global__ void reluBackwardKernel(float *d_dZ, const float *Z, int size) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < size) {
        // Apply the mask: gradient is zeroed out where the pre-activation was <= 0
        if (Z[i] <= 0.0f) {
            d_dZ[i] = 0.0f;
        }
    }
}

/**
 * @brief Performs Stochastic Gradient Descent (SGD) update: W = W - LR * dW
 */
__global__ void updateWeightsKernel(float *W, const float *dW, int size, float lr) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < size) {
        W[i] -= lr * dW[i];
    }
}

// --- MAIN FUNCTION (Simulating the Backward Pass Data Flow) ---

int main() {
    std::cout << "--- Backward Pass Kernel Demonstration ---" << std::endl;
    std::cout << "Simulating data flow for M=" << M << ", D_hid=" << DHID << ", C=" << C << std::endl;

    // --- 1. Memory Allocation and Dummy Data (Simulate Forward Pass Outputs) ---
    float *d_W2, *d_B2, *d_A1, *d_Z1_pre, *d_dZ2_grad;
    int *d_Y_true;

    // Layer 2 Tensors (Required for L2 Backward)
    CUDA_CHECK(cudaMalloc((void**)&d_W2, DHID * C * sizeof(float)));
    CUDA_CHECK(cudaMalloc((void**)&d_B2, C * sizeof(float)));
    CUDA_CHECK(cudaMalloc((void**)&d_dZ2_grad, M * C * sizeof(float))); // Input gradient

    // Layer 1 Tensors (Required for L1 Backward)
    CUDA_CHECK(cudaMalloc((void**)&d_A1, M * DHID * sizeof(float))); // Input to L2 GEMM
    CUDA_CHECK(cudaMalloc((void**)&d_Z1_pre, M * DHID * sizeof(float))); // L1 Pre-ReLU
    
    // Gradients to be calculated
    float *d_dW2, *d_dB2, *d_dA1_grad, *d_dW1, *d_dB1;
    CUDA_CHECK(cudaMalloc((void**)&d_dW2, DHID * C * sizeof(float)));
    CUDA_CHECK(cudaMalloc((void**)&d_dB2, C * sizeof(float)));
    CUDA_CHECK(cudaMalloc((void**)&d_dA1_grad, M * DHID * sizeof(float)));
    CUDA_CHECK(cudaMalloc((void**)&d_dW1, DIN * DHID * sizeof(float)));
    CUDA_CHECK(cudaMalloc((void**)&d_dB1, DHID * sizeof(float)));

    // Dummy Initialization (Fill with arbitrary values to simulate forward pass)
    // In a real scenario, these would be outputs of the forward pass
    std::vector<int> h_Y_true(M);
    std::vector<float> h_Y_hat(M * C);
    for (int i = 0; i < M * C; ++i) h_Y_hat[i] = (float)rand() / RAND_MAX;
    for (int i = 0; i < M; ++i) h_Y_true[i] = rand() % C;
    CUDA_CHECK(cudaMalloc((void**)&d_Y_true, M * sizeof(int)));
    CUDA_CHECK(cudaMemcpy(d_Y_true, h_Y_true.data(), M * sizeof(int), cudaMemcpyHostToDevice));
    // Y_hat is copied into d_dZ2_grad to simulate the first step
    CUDA_CHECK(cudaMemcpy(d_dZ2_grad, h_Y_hat.data(), M * C * sizeof(float), cudaMemcpyHostToDevice));
    
    // --- 2. BACKWARD PASS CHAIN EXECUTION (Simplified) ---
    
    // Grid/Block Setup
    dim3 grid_mat(DIV_UP(C, 16), DIV_UP(M, 16));
    dim3 block_mat(16, 16);
    int threads_bias = THREADS_PER_BLOCK;
    int blocks_bias_c = DIV_UP(C, THREADS_PER_BLOCK);
    int blocks_bias_hid = DIV_UP(DHID, THREADS_PER_BLOCK);
    
    std::cout << "\n[L2] 1. Initial Gradient (dL/dZ2 = Y_hat - Y_true)..." << std::endl;
    // d_dZ2_grad is updated in-place
    lossBackwardKernel<<<grid_mat, block_mat>>>(d_dZ2_grad, d_Y_true, d_dZ2_grad, M, C);

    std::cout << "[L2] 2. Bias Gradient (dL/dB2)..." << std::endl;
    biasBackwardKernel<<<blocks_bias_c, threads_bias>>>(d_dZ2_grad, d_dB2, M, C);

    std::cout << "[L2] 3. Weight Gradient (dL/dW2)..." << std::endl;
    dim3 grid_w2(DIV_UP(C, 16), DIV_UP(DHID, 16));
    gemmBackwardWeightKernel<<<grid_w2, block_mat>>>(d_A1, d_dZ2_grad, d_dW2, M, DHID, C);

    std::cout << "[L2] 4. Input Gradient (dL/dA1) to be passed to L1..." << std::endl;
    dim3 grid_a1(DIV_UP(DHID, 16), DIV_UP(M, 16));
    gemmBackwardInputKernel<<<grid_a1, block_mat>>>(d_dZ2_grad, d_W2, d_dA1_grad, M, DHID, C);

    // Continue to L1 (just showing the start of the chain)

    std::cout << "[L1] 5. ReLU Backward (dL/dZ1 = dL/dA1 * ReLU'(Z1_pre))..." << std::endl;
    int size1 = M * DHID;
    reluBackwardKernel<<<DIV_UP(size1, THREADS_PER_BLOCK), THREADS_PER_BLOCK>>>(d_dA1_grad, d_Z1_pre, size1);

    std::cout << "\n[Optimizer] Update kernels (W = W - LR * dW) are ready." << std::endl;

    CUDA_CHECK(cudaDeviceSynchronize());
    std::cout << "\n--- Backward Pass Kernels executed successfully. ---" << std::endl;

    // Cleanup
    cudaFree(d_W2); cudaFree(d_B2); cudaFree(d_A1); cudaFree(d_Z1_pre); cudaFree(d_dZ2_grad);
    cudaFree(d_dW2); cudaFree(d_dB2); cudaFree(d_dA1_grad); cudaFree(d_dW1); cudaFree(d_dB1);
    cudaFree(d_Y_true);

    return 0;
}
EOF

nvcc -std=c++17 backward_pass.cu -o backward_pass_output && ./backward_pass_output

backward_pass.cu(181): warning #177-D: variable "blocks_bias_hid" was declared but never referenced

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

backward_pass.cu(11): warning #177-D: variable "LR" was declared but never referenced



--- Backward Pass Kernel Demonstration ---
Simulating data flow for M=128, D_hid=32, C=10

[L2] 1. Initial Gradient (dL/dZ2 = Y_hat - Y_true)...
[L2] 2. Bias Gradient (dL/dB2)...
[L2] 3. Weight Gradient (dL/dW2)...
[L2] 4. Input Gradient (dL/dA1) to be passed to L1...
[L1] 5. ReLU Backward (dL/dZ1 = dL/dA1 * ReLU'(Z1_pre))...

[Optimizer] Update kernels (W = W - LR * dW) are ready.

--- Backward Pass Kernels executed successfully. ---


In [19]:
%%bash
export CUDA_HOME=/packages/apps/spack/21/opt/spack/linux-rocky8-zen3/gcc-12.1.0/cuda-12.0.1-x3bnvayrybncl3rqu6zk4zzu4oztblqi
export PATH=$CUDA_HOME/bin:$PATH
export LD_LIBRARY_PATH=$CUDA_HOME/lib64:$LD_LIBRARY_PATH
cat > full_loop.cu <<'EOF'

#include <iostream>
#include <vector>
#include <cmath>
#include <numeric>

// Global constants for the network size
const int M = 128;   // Batch Size (Rows)
const int DIN = 64;  // Input Dimension
const int DHID = 32; // Hidden Dimension Size (used for both hidden layers)
const int C = 10;    // Number of Classes (Output Dimension)
const float LR = 0.01f; // Learning Rate

const int THREADS_PER_BLOCK = 256;

// Helper macros
#define CUDA_CHECK(call)                                                          \
{                                                                                 \
    cudaError_t err = call;                                                       \
    if (err != cudaSuccess) {                                                     \
        fprintf(stderr, "CUDA error at %s:%d code=%d(%s) \"%s\"\n",               \
                __FILE__, __LINE__, err, cudaGetErrorString(err), #call);         \
        exit(EXIT_FAILURE);                                                       \
    }                                                                             \
}
#define DIV_UP(a, b) ((a + b - 1) / b)

// --- UTILITY KERNELS ---

/**
 * @brief Initializes a device vector with random numbers between -0.5 and 0.5.
 */
__global__ void initWeightsKernel(float *d_data, int size) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < size) {
        unsigned int seed = i;
        float rand_val = (float)seed / (float)0xFFFFFFFF; 
        d_data[i] = (rand_val - 0.5f); 
    }
}

/**
 * @brief Performs a generic parallel sum reduction.
 */
__global__ void reduceSumKernel(float *d_in, float *d_out, int size) {
    __shared__ float sdata[THREADS_PER_BLOCK];
    
    int tid = threadIdx.x;
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    
    sdata[tid] = (i < size) ? d_in[i] : 0.0f;
    __syncthreads();

    for (int s = blockDim.x / 2; s > 0; s >>= 1) {
        if (tid < s) {
            sdata[tid] += sdata[tid + s];
        }
        __syncthreads();
    }
    
    if (tid == 0) {
        atomicAdd(d_out, sdata[0]); 
    }
}

// --- FORWARD PASS KERNELS ---

/**
 * @brief Performs Matrix Multiplication and Bias Addition: Z = X * W + B.
 */
__global__ void simpleGemmBiasKernel(const float *X, const float *W, const float *B, float *Z, 
                                     int M, int K, int N) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < M && col < N) {
        float sum = 0.0f;
        for (int k = 0; k < K; ++k) {
            sum += X[row * K + k] * W[k * N + col];
        }
        sum += B[col];
        Z[row * N + col] = sum;
    }
}

/**
 * @brief Performs element-wise ReLU activation: A = max(0, Z).
 */
__global__ void reluKernel(float *A, int size) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < size) {
        if (A[i] < 0.0f) {
            A[i] = 0.0f;
        }
    }
}

/**
 * @brief Performs row-wise Softmax activation.
 */
__global__ void softmaxKernel(float *Z, int M, int C) {
    int row = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < M) {
        float max_val = -1e20f;
        float sum_exp = 0.0f;
        int row_start = row * C;
        
        for (int j = 0; j < C; ++j) {
            max_val = fmaxf(max_val, Z[row_start + j]);
        }

        for (int j = 0; j < C; ++j) {
            Z[row_start + j] = expf(Z[row_start + j] - max_val);
            sum_exp += Z[row_start + j];
        }

        for (int j = 0; j < C; ++j) {
            Z[row_start + j] /= sum_exp;
        }
    }
}

/**
 * @brief Calculates the Cross-Entropy Loss element-wise.
 */
__global__ void crossEntropyKernel(const float *Y_hat, const int *Y_true, float *d_loss_elements, 
                                   int M, int C) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;

    if (i < M) {
        int true_class = Y_true[i];
        float predicted_prob = Y_hat[i * C + true_class];
        float loss = -logf(fmaxf(predicted_prob, 1e-9f)); 
        d_loss_elements[i] = loss;
    }
}

// --- BACKWARD PASS KERNELS (REUSED GENERIC FUNCTIONS) ---

/**
 * @brief Calculates the initial gradient dL/dZ (Softmax + Cross-Entropy fusion).
 */
__global__ void lossBackwardKernel(const float *Y_hat, const int *Y_true, float *d_dZ_out, 
                                   int M, int C) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < M && col < C) {
        float gradient = Y_hat[row * C + col]; 
        if (col == Y_true[row]) {
            gradient -= 1.0f;
        }
        // Normalize by Batch Size (M)
        d_dZ_out[row * C + col] = gradient / (float)M; 
    }
}

/**
 * @brief Calculates the gradient passed to the previous layer: dL/dX = dL/dZ * W^T
 */
__global__ void gemmBackwardInputKernel(const float *d_dZ, const float *W, float *d_dX, 
                                        int M, int K, int N) {
    // d_dX dimensions: M x K
    // d_dZ dimensions: M x N
    // W dimensions: K x N
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < M && col < K) {
        float sum = 0.0f;
        for (int n = 0; n < N; ++n) {
            // W[col * N + n] performs implicit transpose on W (originally K x N)
            sum += d_dZ[row * N + n] * W[col * N + n];
        }
        d_dX[row * K + col] = sum;
    }
}

/**
 * @brief Calculates the weight gradient: dL/dW = X^T * dL/dZ
 */
__global__ void gemmBackwardWeightKernel(const float *X, const float *d_dZ, float *d_dW, 
                                         int M, int K, int N) {
    // d_dW dimensions: K x N
    // X dimensions: M x K
    // d_dZ dimensions: M x N
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < K && col < N) {
        float sum = 0.0f;
        for (int m = 0; m < M; ++m) {
            // X[m * K + row] performs implicit transpose on X (originally M x K)
            sum += X[m * K + row] * d_dZ[m * N + col];
        }
        d_dW[row * N + col] = sum;
    }
}

/**
 * @brief Performs the backward pass for Bias: dL/dB = sum(dL/dZ, axis=0)
 */
__global__ void biasBackwardKernel(const float *d_dZ, float *d_dB, int M, int N) {
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (col < N) {
        float sum = 0.0f;
        for (int m = 0; m < M; ++m) {
            sum += d_dZ[m * N + col];
        }
        d_dB[col] = sum;
    }
}

/**
 * @brief Performs element-wise ReLU backward: dL/dZ = dL/dA * ReLU'(Z)
 */
__global__ void reluBackwardKernel(float *d_dZ, const float *Z, int size) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < size) {
        // Sets gradient to zero where the forward pass pre-activation was <= 0
        if (Z[i] <= 0.0f) {
            d_dZ[i] = 0.0f;
        }
    }
}

/**
 * @brief Performs Stochastic Gradient Descent (SGD) update: W = W - LR * dW
 */
__global__ void updateWeightsKernel(float *W, const float *dW, int size, float lr) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < size) {
        W[i] -= lr * dW[i];
    }
}


// --- MAIN FUNCTION: Orchestrating the Full Network ---

int main() {
    std::cout << "--- Integrated Full Training Step on GPU (3 Layers) ---" << std::endl;
    std::cout << "Configuration: M=" << M << ", D_in=" << DIN << ", D_hid=" << DHID << ", C=" << C << ", LR=" << LR << std::endl;

    // --- 1. Memory Setup and Allocation (3 LAYERS) ---
    // Layer 1: DIN -> DHID
    float *d_W1, *d_B1, *d_Z1_pre, *d_A1; 
    // Layer 2: DHID -> DHID
    float *d_W2, *d_B2, *d_Z2_pre, *d_A2; 
    // Layer 3 (Output): DHID -> C
    float *d_W3, *d_B3, *d_Z3; // d_Z3 holds Y_hat after Softmax
    
    float *d_X;
    int *d_Y_true;
    float *d_loss_elements, *d_final_loss_sum; 
    
    // Gradients
    float *d_dW1, *d_dB1, *d_dA1_grad;
    float *d_dW2, *d_dB2, *d_dA2_grad;
    float *d_dW3, *d_dB3, *d_dZ3_grad;

    std::vector<int> h_Y_true(M);
    std::vector<float> h_X(M * DIN);
    for (int i = 0; i < M * DIN; ++i) h_X[i] = (float)rand() / RAND_MAX;
    for (int i = 0; i < M; ++i) h_Y_true[i] = rand() % C;
    
    // --- Allocate Tensors ---
    CUDA_CHECK(cudaMalloc((void**)&d_X, M * DIN * sizeof(float)));
    CUDA_CHECK(cudaMalloc((void**)&d_Y_true, M * sizeof(int)));
    
    // L1 Tensors (DIN -> DHID)
    CUDA_CHECK(cudaMalloc((void**)&d_W1, DIN * DHID * sizeof(float)));
    CUDA_CHECK(cudaMalloc((void**)&d_B1, DHID * sizeof(float)));
    CUDA_CHECK(cudaMalloc((void**)&d_Z1_pre, M * DHID * sizeof(float))); 
    CUDA_CHECK(cudaMalloc((void**)&d_A1, M * DHID * sizeof(float)));
    CUDA_CHECK(cudaMalloc((void**)&d_dW1, DIN * DHID * sizeof(float)));
    CUDA_CHECK(cudaMalloc((void**)&d_dB1, DHID * sizeof(float)));
    CUDA_CHECK(cudaMalloc((void**)&d_dA1_grad, M * DHID * sizeof(float))); // dL/dA1, reused as dL/dZ1
    
    // L2 Tensors (DHID -> DHID)
    CUDA_CHECK(cudaMalloc((void**)&d_W2, DHID * DHID * sizeof(float)));
    CUDA_CHECK(cudaMalloc((void**)&d_B2, DHID * sizeof(float)));
    CUDA_CHECK(cudaMalloc((void**)&d_Z2_pre, M * DHID * sizeof(float))); 
    CUDA_CHECK(cudaMalloc((void**)&d_A2, M * DHID * sizeof(float)));
    CUDA_CHECK(cudaMalloc((void**)&d_dW2, DHID * DHID * sizeof(float)));
    CUDA_CHECK(cudaMalloc((void**)&d_dB2, DHID * sizeof(float)));
    CUDA_CHECK(cudaMalloc((void**)&d_dA2_grad, M * DHID * sizeof(float))); // dL/dA2, reused as dL/dZ2

    // L3 Tensors (DHID -> C)
    CUDA_CHECK(cudaMalloc((void**)&d_W3, DHID * C * sizeof(float)));
    CUDA_CHECK(cudaMalloc((void**)&d_B3, C * sizeof(float)));
    CUDA_CHECK(cudaMalloc((void**)&d_Z3, M * C * sizeof(float))); 
    CUDA_CHECK(cudaMalloc((void**)&d_dW3, DHID * C * sizeof(float)));
    CUDA_CHECK(cudaMalloc((void**)&d_dB3, C * sizeof(float)));
    CUDA_CHECK(cudaMalloc((void**)&d_dZ3_grad, M * C * sizeof(float)));

    // Loss Tensors
    CUDA_CHECK(cudaMalloc((void**)&d_loss_elements, M * sizeof(float)));
    CUDA_CHECK(cudaMalloc((void**)&d_final_loss_sum, sizeof(float)));

    // Initial Data Transfer and Initialization
    CUDA_CHECK(cudaMemcpy(d_X, h_X.data(), M * DIN * sizeof(float), cudaMemcpyHostToDevice));
    CUDA_CHECK(cudaMemcpy(d_Y_true, h_Y_true.data(), M * sizeof(int), cudaMemcpyHostToDevice));
    float zero = 0.0f;
    CUDA_CHECK(cudaMemcpy(d_final_loss_sum, &zero, sizeof(float), cudaMemcpyHostToDevice));




    // Initialize Weights
    initWeightsKernel<<<DIV_UP(DIN * DHID, THREADS_PER_BLOCK), THREADS_PER_BLOCK>>>(d_W1, DIN * DHID);
    initWeightsKernel<<<DIV_UP(DHID, THREADS_PER_BLOCK), THREADS_PER_BLOCK>>>(d_B1, DHID);
    initWeightsKernel<<<DIV_UP(DHID * DHID, THREADS_PER_BLOCK), THREADS_PER_BLOCK>>>(d_W2, DHID * DHID);
    initWeightsKernel<<<DIV_UP(DHID, THREADS_PER_BLOCK), THREADS_PER_BLOCK>>>(d_B2, DHID);
    initWeightsKernel<<<DIV_UP(DHID * C, THREADS_PER_BLOCK), THREADS_PER_BLOCK>>>(d_W3, DHID * C);
    initWeightsKernel<<<DIV_UP(C, THREADS_PER_BLOCK), THREADS_PER_BLOCK>>>(d_B3, C);
    
    // --- 2. FORWARD PASS EXECUTION (3 LAYERS) ---
    std::cout << "\n--- FORWARD PASS ---" << std::endl;
    dim3 block_mat(16, 16);
    int size_hid = M * DHID;
    
    // --- L1: Input (DIN) -> Hidden (DHID) ---
    dim3 grid_mat1(DIV_UP(DHID, 16), DIV_UP(M, 16));
    simpleGemmBiasKernel<<<grid_mat1, block_mat>>>(d_X, d_W1, d_B1, d_Z1_pre, M, DIN, DHID);
    CUDA_CHECK(cudaMemcpy(d_A1, d_Z1_pre, size_hid * sizeof(float), cudaMemcpyDeviceToDevice));
    reluKernel<<<DIV_UP(size_hid, THREADS_PER_BLOCK), THREADS_PER_BLOCK>>>(d_A1, size_hid);

    // --- L2: Hidden (DHID) -> Hidden (DHID) ---
    // Kernel uses DHID as K and N
    simpleGemmBiasKernel<<<grid_mat1, block_mat>>>(d_A1, d_W2, d_B2, d_Z2_pre, M, DHID, DHID);
    CUDA_CHECK(cudaMemcpy(d_A2, d_Z2_pre, size_hid * sizeof(float), cudaMemcpyDeviceToDevice));
    reluKernel<<<DIV_UP(size_hid, THREADS_PER_BLOCK), THREADS_PER_BLOCK>>>(d_A2, size_hid);

    // --- L3: Hidden (DHID) -> Output (C) ---
    dim3 grid_mat3(DIV_UP(C, 16), DIV_UP(M, 16));
    simpleGemmBiasKernel<<<grid_mat3, block_mat>>>(d_A2, d_W3, d_B3, d_Z3, M, DHID, C);
    softmaxKernel<<<DIV_UP(M, THREADS_PER_BLOCK), THREADS_PER_BLOCK>>>(d_Z3, M, C); // d_Z3 is now Y_hat

    // LOSS CALCULATION
    crossEntropyKernel<<<DIV_UP(M, THREADS_PER_BLOCK), THREADS_PER_BLOCK>>>(d_Z3, d_Y_true, d_loss_elements, M, C);
    reduceSumKernel<<<DIV_UP(M, THREADS_PER_BLOCK), THREADS_PER_BLOCK>>>(d_loss_elements, d_final_loss_sum, M);

    // --- 3. BACKWARD PASS EXECUTION (3 LAYERS) ---
    std::cout << "\n--- BACKWARD PASS ---" << std::endl;

    // --- L3 BACKWARD (Output Layer: DHID -> C) ---
    // Step 1: Calculate dL/dZ3 = Y_hat - Y_true
    lossBackwardKernel<<<grid_mat3, block_mat>>>(d_Z3, d_Y_true, d_dZ3_grad, M, C);

    // Step 2: Calculate dW3 and dB3 (Inputs: A2, dZ3_grad)
    biasBackwardKernel<<<DIV_UP(C, THREADS_PER_BLOCK), THREADS_PER_BLOCK>>>(d_dZ3_grad, d_dB3, M, C);
    dim3 grid_w3(DIV_UP(C, 16), DIV_UP(DHID, 16));
    gemmBackwardWeightKernel<<<grid_w3, block_mat>>>(d_A2, d_dZ3_grad, d_dW3, M, DHID, C); // X=A2, K=DHID, N=C

    // Step 3: Calculate dL/dA2 (gradient to pass to L2)
    dim3 grid_a2(DIV_UP(DHID, 16), DIV_UP(M, 16));
    gemmBackwardInputKernel<<<grid_a2, block_mat>>>(d_dZ3_grad, d_W3, d_dA2_grad, M, DHID, C); // K=DHID, N=C

    // --- L2 BACKWARD (Hidden Layer: DHID -> DHID) ---
    // Step 4: ReLU Backward on dL/dA2 to get dL/dZ2 in-place (d_dA2_grad now holds dL/dZ2)
    reluBackwardKernel<<<DIV_UP(size_hid, THREADS_PER_BLOCK), THREADS_PER_BLOCK>>>(d_dA2_grad, d_Z2_pre, size_hid); 

    // Step 5: Calculate dW2 and dB2 (Inputs: A1, dL/dZ2 in d_dA2_grad)
    biasBackwardKernel<<<DIV_UP(DHID, THREADS_PER_BLOCK), THREADS_PER_BLOCK>>>(d_dA2_grad, d_dB2, M, DHID);
    dim3 grid_w2(DIV_UP(DHID, 16), DIV_UP(DHID, 16));
    gemmBackwardWeightKernel<<<grid_w2, block_mat>>>(d_A1, d_dA2_grad, d_dW2, M, DHID, DHID); // X=A1, K=DHID, N=DHID

    // Step 6: Calculate dL/dA1 (gradient to pass to L1)
    dim3 grid_a1(DIV_UP(DHID, 16), DIV_UP(M, 16));
    gemmBackwardInputKernel<<<grid_a1, block_mat>>>(d_dA2_grad, d_W2, d_dA1_grad, M, DHID, DHID); // K=DHID, N=DHID

    // --- L1 BACKWARD (Hidden Layer: DIN -> DHID) ---
    // Step 7: ReLU Backward on dL/dA1 to get dL/dZ1 in-place (d_dA1_grad now holds dL/dZ1)
    reluBackwardKernel<<<DIV_UP(size_hid, THREADS_PER_BLOCK), THREADS_PER_BLOCK>>>(d_dA1_grad, d_Z1_pre, size_hid); 

    // Step 8: Calculate dW1 and dB1 (Inputs: X, dL/dZ1 in d_dA1_grad)
    biasBackwardKernel<<<DIV_UP(DHID, THREADS_PER_BLOCK), THREADS_PER_BLOCK>>>(d_dA1_grad, d_dB1, M, DHID);
    dim3 grid_w1(DIV_UP(DHID, 16), DIV_UP(DIN, 16));
    gemmBackwardWeightKernel<<<grid_w1, block_mat>>>(d_X, d_dA1_grad, d_dW1, M, DIN, DHID); // X=d_X, K=DIN, N=DHID

    // --- 4. OPTIMIZER STEP (SGD) ---
    std::cout << "\n--- OPTIMIZER STEP (SGD) ---" << std::endl;
    // L3 Update
    updateWeightsKernel<<<DIV_UP(DHID * C, THREADS_PER_BLOCK), THREADS_PER_BLOCK>>>(d_W3, d_dW3, DHID * C, LR);
    updateWeightsKernel<<<DIV_UP(C, THREADS_PER_BLOCK), THREADS_PER_BLOCK>>>(d_B3, d_dB3, C, LR);
    // L2 Update
    updateWeightsKernel<<<DIV_UP(DHID * DHID, THREADS_PER_BLOCK), THREADS_PER_BLOCK>>>(d_W2, d_dW2, DHID * DHID, LR);
    updateWeightsKernel<<<DIV_UP(DHID, THREADS_PER_BLOCK), THREADS_PER_BLOCK>>>(d_B2, d_dB2, DHID, LR);
    // L1 Update
    updateWeightsKernel<<<DIV_UP(DIN * DHID, THREADS_PER_BLOCK), THREADS_PER_BLOCK>>>(d_W1, d_dW1, DIN * DHID, LR);
    updateWeightsKernel<<<DIV_UP(DHID, THREADS_PER_BLOCK), THREADS_PER_BLOCK>>>(d_B1, d_dB1, DHID, LR);

    // --- 5. Final Output ---
    CUDA_CHECK(cudaDeviceSynchronize()); 
    float final_loss_sum_h;
    CUDA_CHECK(cudaMemcpy(&final_loss_sum_h, d_final_loss_sum, sizeof(float), cudaMemcpyDeviceToHost));
    float avg_loss = final_loss_sum_h / (float)M;

    std::cout << "\n--- Training Step Complete ---" << std::endl;
    std::cout << "Average Cross-Entropy Loss: " << avg_loss << std::endl;
    
    // Cleanup (All 3 layers)
    cudaFree(d_X); cudaFree(d_Y_true); cudaFree(d_loss_elements); cudaFree(d_final_loss_sum);
    cudaFree(d_W1); cudaFree(d_B1); cudaFree(d_Z1_pre); cudaFree(d_A1);
    cudaFree(d_W2); cudaFree(d_B2); cudaFree(d_Z2_pre); cudaFree(d_A2);
    cudaFree(d_W3); cudaFree(d_B3); cudaFree(d_Z3); 
    cudaFree(d_dW1); cudaFree(d_dB1); cudaFree(d_dA1_grad); 
    cudaFree(d_dW2); cudaFree(d_dB2); cudaFree(d_dA2_grad); 
    cudaFree(d_dW3); cudaFree(d_dB3); cudaFree(d_dZ3_grad);

    return 0;
}
EOF

nvcc -std=c++17 full_loop.cu -o full_loop_output && ./full_loop_output

--- Integrated Full Training Step on GPU (3 Layers) ---
Configuration: M=128, D_in=64, D_hid=32, C=10, LR=0.01

--- FORWARD PASS ---

--- BACKWARD PASS ---

--- OPTIMIZER STEP (SGD) ---

--- Training Step Complete ---
Average Cross-Entropy Loss: 2.30259
